In [ ]:
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import os

# load environment variables from .env file for project
dotenv_path = Path('../.env')
load_dotenv(dotenv_path=dotenv_path)

data_directory = os.getenv("OUTPUT_DIRECTORY")

In [ ]:
os.listdir(data_directory)

In [ ]:
excel_files = [filename for filename in os.listdir(data_directory) if filename.endswith(".xlsx")]
excel_files

In [ ]:
ah_receipts = os.path.join(data_directory, excel_files[0])
ah_receipts

In [ ]:
from openpyxl import load_workbook

ah_workbook = load_workbook(filename=ah_receipts)
ah_sheet_names = ah_workbook.sheetnames
ah_sheet_names

In [ ]:
coicop_sheets = {sheet: sheet.lower().replace("coi", "") for sheet in ah_sheet_names if sheet.lower().startswith("coi")}
coicop_sheets

In [ ]:
for sheet_name, supermarket_name in coicop_sheets.items():
    ah_receipts_df = pd.read_excel(ah_receipts, sheet_name=sheet_name)
    ah_receipts_df = ah_receipts_df.rename(columns={'Kassabonomschrijving': 'receipt_text', 'ArtikelEAN': "ean_number", 'IsbaOmschrijving': 'isba_description', 'BG': 'supermarket_id', 'Coicop': 'coicop_number'})
    ah_receipts_df = ah_receipts_df.rename(columns={column_name: column_name.lower() for column_name in ah_receipts_df.columns})
    ah_receipts_df.to_parquet(os.path.join(data_directory, f"{supermarket_name}_receipts.parquet"), engine="pyarrow")  

In [ ]:
ah_receipts_filenames = [os.path.join(data_directory, filename) for filename in os.listdir(data_directory) if filename.startswith("ah") and filename.endswith(".parquet")]

for ah_receipts_filename in ah_receipts_filenames:
    ah_receipts_df = pd.read_parquet(ah_receipts_filename, engine="pyarrow")
    print(ah_receipts_df.head())